<a href="https://colab.research.google.com/github/mindyng/Projects/blob/master/BERT_Using_Tweets_to_Determine_Real_or_Fake_Disaster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Toxic Comment Classification Challenge

 Inspired by: https://www.kaggle.com/nkaenzig/bert-tensorflow-2-huggingface-transformers



In [1]:
!pip install transformers

     |████████████████████████████████| 675kB 6.6MB/s 
     |████████████████████████████████| 890kB 18.0MB/s 
     |████████████████████████████████| 3.8MB 38.3MB/s 
     |████████████████████████████████| 1.1MB 41.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=16d6d2685c31c884cde11286cc3fb7cf24ef684621a225bb9272f8071fcdb4c7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm

from transformers import BertTokenizer, BertModel
import os

Below are the steps involved with any transformer.

# Preprocessing

### 1. Tokenize input data and other input details such as Attention Mask for BERT so that attention on padded sequences is not ignored.

### 2. Convert tokens to input ID sequences.

### 3. Pad IDs to a fixed length. 

# Modeling

### 1. Load model and feed in input ID sequence (batches are best especially when there is limited CPU)

### 2. Get output of last hidden layer. This layer has sequence representation embedding at 0 index. So we the output is called last_hidden_layer[0]. 

### 3. Embeddings can be used as inputs for different ML/DL models.

Using BERT Transformers.

In [3]:
model_type = 'bert-base-uncased'
max_size = 150
batch_size = 200

In [4]:
train_df = pd.read_csv("/content/drive/My Drive/train.csv")
test_df = pd.read_csv("/content/drive/My Drive/test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
test_df

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


## Load Tokenizer and Model

In [6]:
tokenizer = BertTokenizer.from_pretrained(model_type)
model = BertModel.from_pretrained(model_type)

## Convert Text to Tokens

In [7]:
tokenized_input = train_df['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens = True)))

In [8]:
print(tokenized_input[1])
print("Here 101 -> [CLS] and 102 -> [SEP]")

[101, 3224, 2543, 2379, 2474, 6902, 3351, 21871, 2243, 1012, 2710, 102]
Here 101 -> [CLS] and 102 -> [SEP]


* [CLS] token = classification token

* In between [CLS] and [SEP] = entire sequence embedding. (Need to take token embedding from output layer)

* [SEP] = end of sequence

## Padding sequence to standardize length

In [9]:
padded_tokenized_input = np.array([i + [0]*(max_size-len(i)) for i in tokenized_input.values])

In [10]:
print(padded_tokenized_input[0])

[  101  2256 15616  2024  1996  3114  1997  2023  1001  8372  2089 16455
  9641  2149  2035   102     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0]


Telling BERT to ignore attention on padded inputs.

In [11]:
attention_masks = np.where(padded_tokenized_input !=0,1,0)
print(attention_masks)

[[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]]


In [12]:
input_ids = torch.tensor(padded_tokenized_input)
attention_masks = torch.tensor(attention_masks)

## Get sequence embeddings

In [13]:
all_train_embedding = []

with torch.no_grad():
  for i in tqdm(range(0,len(input_ids),200)):    
    last_hidden_states = model(input_ids[i:min(i+200,len(train_df))], attention_mask = attention_masks[i:min(i+200,len(train_df))])[0][:,0,:].numpy()
    all_train_embedding.append(last_hidden_states)

100%|██████████| 39/39 [57:56<00:00, 89.14s/it]


In [14]:
unbatched_train = []
for batch in all_train_embedding:
    for seq in batch:
        unbatched_train.append(seq)

train_labels = train_df['target']

Train Test Split to be used in Various Models

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test =  train_test_split(unbatched_train, train_labels, test_size=0.33, random_state=42, stratify=train_labels)

Feed these training sets into various models for training. Then fitting model with test data to validate model.